In [23]:
import numpy as np
import scipy as sp

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import path

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
data_path = path.Path(".\\data")
df = pd.read_csv(data_path/"train.csv")

train_x, test_x, train_y, test_y = train_test_split(df.to_numpy()[:, 1:50], df.to_numpy()[:,-1])

In [3]:
model = RandomForestClassifier()
model.fit(train_x, train_y)
model.score(test_x, test_y)

0.5752